# Design Patterns in Python - Detailed Explanation and Implementation



## 3. Behavioral Patterns

### Observer

Defines a one-to-many dependency so that when one object changes state, all its dependents are notified.
Use when: A change to one object requires changing others.

#### Intent
<img src="behaviour1.png" alt="Alt Text" width="600"/>

<img src="behaviour2.png" alt="Alt Text" width="600"/>



#### Structure
<img src="behaviour3.png" alt="Alt Text" width="500"/>

In [1]:



class Subject:
    def __init__(self):
        self._observers = []

    def attach(self, observer):
        self._observers.append(observer)

    def notify(self, msg):
        for observer in self._observers:
            observer.update(msg)

class Observer:
    
    def __init__(self, name):
        self.name = name

    def update(self, msg):
        print(f"Observer {self.name} received: {msg}")

# Usage
subject = Subject()
obs1 = Observer("Observer 1")
obs2 = Observer("Observer 2")
subject.attach(obs1)
subject.attach(obs2)
subject.notify("Event Happened")

Observer Observer 1 received: Event Happened
Observer Observer 2 received: Event Happened


### Strategy
Defines a family of algorithms, encapsulates each one, and makes them interchangeable.
Use when: You need different variants of an algorithm.

#### Intent
<img src="strategy1.png" alt="Alt Text" width="600"/>





#### Structure
<img src="strategy2.png" alt="Alt Text" width="500"/>

In [2]:

class Strategy:
    def execute(self, data): pass

class SortAsc(Strategy):
    def execute(self, data): return sorted(data)

class SortDesc(Strategy):
    def execute(self, data): return sorted(data, reverse=True)

class Context:
    def __init__(self, strategy): self.strategy = strategy
    def sort(self, data): return self.strategy.execute(data)

# Usage
context = Context(SortAsc())
print(context.sort([3, 1, 2]))

[1, 2, 3]


### Command

Encapsulates a request as an object, thereby letting you parameterize clients with different requests.
Use when: You need to issue requests to objects without knowing anything about the operation being requested.

#### Intent
<img src="command1.png" alt="Alt Text" width="600"/>





#### Structure
<img src="command2.png" alt="Alt Text" width="500"/>


In [3]:

class Command:
    def execute(self): pass

class LightOn(Command):
    def execute(self): return "Light is ON"

class Remote:
    def __init__(self): self.command = None
    def set_command(self, cmd): self.command = cmd
    def press(self): return self.command.execute()

# Usage
remote = Remote()
remote.set_command(LightOn())
print(remote.press())

Light is ON


### State

Allows an object to alter its behavior when its internal state changes.
Use when: An object must change its behavior at runtime depending on its state.


In [4]:

class State:
    def handle(self): pass

#HappyState is Group Admin
class HappyState(State):
    def handle(self): return "I'm happy!"

#SadSTate is Group Member
class SadState(State):
    def handle(self): return "I'm sad."

#Context is Whatsapp Group
class Context:
    def __init__(self, state): self.state = state
    def request(self): return self.state.handle()

# Usage
context = Context(HappyState())
print(context.request())

I'm happy!


### Chain of Responsibility

Gives more than one object a chance to handle a request by passing it along the chain.
Use when: More than one object may handle a request.


In [5]:

class Handler:
    def __init__(self, successor=None):
        self.successor = successor

    def handle(self, request):
        if self.successor:
            return self.successor.handle(request)
        return "End of chain"

class ConcreteHandlerA(Handler):
    def handle(self, request):
        if request == 'A':
            return "Handled by A"
        return super().handle(request)

class ConcreteHandlerB(Handler):
    def handle(self, request):
        if request == 'B':
            return "Handled by B"
        return super().handle(request)

# Usage
chain = ConcreteHandlerA(ConcreteHandlerB())
print(chain.handle('B'))
print(chain.handle('C'))

Handled by B
End of chain


In [7]:
from abc import ABC, abstractmethod

# Handler Interface
class Handler(ABC):
    def __init__(self):
        self.next_handler = None

    def set_next(self, handler):
        self.next_handler = handler
        return handler  # Enables chaining

    @abstractmethod
    def handle(self, level, message):
        pass


# Concrete Handlers
class ConsoleLogger(Handler):
    def handle(self, level, message):
        print("ConsoleLogger")
        if level == "INFO":
            print(f"Exists : [Console] {message}")
        elif self.next_handler:
            self.next_handler.handle(level, message)


class FileLogger(Handler):
    def handle(self, level, message):
        print("FileLogger")
        if level == "WARNING":
            print(f"[File] {message} (logged to file)")
        elif self.next_handler:
            self.next_handler.handle(level, message)


class EmailLogger(Handler):
    def handle(self, level, message):
        print("EmailLogger")
        if level == "ERROR":
            print(f"[Email] {message} (sent via email)")
        elif self.next_handler:
            self.next_handler.handle(level, message)


# Setup chain: Console -> File -> Email
console = ConsoleLogger()
file = FileLogger()
email = EmailLogger()

console.set_next(file).set_next(email)

# Sample requests
console.handle("INFO", "This is an info message.")
console.handle("WARNING", "This is a warning message.")
console.handle("ERROR", "This is an error message.")
console.handle("DEBUG", "This is a debug message.")  # Not handled


ConsoleLogger
Exists : [Console] This is an info message.
ConsoleLogger
FileLogger
[File] This is a warning message. (logged to file)
ConsoleLogger
FileLogger
EmailLogger
[Email] This is an error message. (sent via email)
ConsoleLogger
FileLogger
EmailLogger
